# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

---
## 📦 Installation

First, let's install all required packages. Run this cell once at the beginning.

In [1]:
# Install required packages
import sys
import platform

# Check if running on macOS with Apple Silicon (M1/M2/M3)
if sys.platform == 'darwin' and platform.processor() == 'arm':
    print("🍎 Detected macOS with Apple Silicon - Installing TensorFlow for Mac...")
    %pip install -q tensorflow-macos tensorflow-metal pandas numpy scikit-learn matplotlib seaborn
else:
    print("💻 Installing TensorFlow...")
    %pip install -q tensorflow pandas numpy scikit-learn matplotlib seaborn

print("✅ Installation complete!")

💻 Installing TensorFlow...
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
Note: you may need to restart the kernel to use updated packages.
✅ Installation complete!


---
## 📚 Imports

Import all necessary libraries.

In [5]:
# Data manipulation
import pandas as pd
import numpy as np

# Machine Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ All imports successful!")

ModuleNotFoundError: No module named 'tensorflow'

---
## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

### 1.1 - Load the Dataset

In [ ]:
# Read the CSV file
df = pd.read_csv('tic-tac-toe.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

### 1.2 - Inspect the Dataset

In [ ]:
# Display dataset info
print("Dataset Information:")
print(df.info())

print("\n" + "="*50)
print("Unique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

print("\n" + "="*50)
print("Value counts for target variable (class):")
print(df['class'].value_counts())

print("\n" + "="*50)
print("Checking for missing values:")
print(df.isnull().sum())

### 1.3 - Convert Categorical to Numeric

In [ ]:
# Create a copy to avoid modifying original
df_encoded = df.copy()

# Encode the 9 grid columns (x, o, b -> numeric)
label_encoder = LabelEncoder()

# Encode each feature column
for col in df_encoded.columns[:-1]:  # All columns except 'class'
    df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

# Encode the target column (class: True/False -> 1/0)
df_encoded['class'] = df_encoded['class'].map({True: 1, False: 0})

print("Encoded dataset:")
print(df_encoded.head())

print("\nData types after encoding:")
print(df_encoded.dtypes)

### 1.4 - Separate Features (X) and Target (y)

In [ ]:
# Separate features and target
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures (X):")
print(X.head())
print(f"\nTarget (y):")
print(y.head())

### 1.5 - Normalize the Input Data

In [ ]:
# Normalize features using StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Convert back to DataFrame for easier viewing
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

print("Normalized features:")
print(X_normalized.head())
print(f"\nMean of normalized features: {X_normalized.mean().mean():.4f}")
print(f"Std of normalized features: {X_normalized.std().mean():.4f}")

---
## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.keras`.

### 2.1 - Split Training and Test Data

In [ ]:
# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"\nFeatures shape: {X_train.shape[1]}")
print(f"\nClass distribution in training set:")
print(y_train.value_counts())

### 2.2 - Create Sequential Model

In [ ]:
# Create a Sequential model
model = Sequential([
    # Input layer + First hidden layer
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    
    # Second hidden layer
    Dense(32, activation='relu'),
    
    # Third hidden layer
    Dense(16, activation='relu'),
    
    # Output layer (2 classes: win or not win)
    Dense(2, activation='softmax')
])

# Display model architecture
print("Model Architecture:")
model.summary()

### 2.3 - Compile the Model

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled successfully!")

### 2.4 - Train the Model

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\n✅ Model training completed!")

### 2.5 - Visualize Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.6 - Evaluate the Model

In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("="*50)
print("MODEL EVALUATION RESULTS")
print("="*50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("="*50)

if test_loss < 0.1 and test_accuracy > 0.95:
    print("✅ Great! Model achieved low loss (<0.1) and high accuracy (>0.95)")
else:
    print("⚠️  Model could be improved (see Step 4)")

### 2.7 - Save the Model

In [ ]:
# Save the model
model.save('tic-tac-toe.keras')
print("✅ Model saved as 'tic-tac-toe.keras'")

---
## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

### 3.1 - Load the Saved Model

In [ ]:
# Load the saved model
loaded_model = load_model('tic-tac-toe.keras')
print("✅ Model loaded successfully!")

### 3.2 - Make Predictions on Random Samples

In [ ]:
# Select 10 random samples from test set
n_samples = 10
random_indices = np.random.choice(X_test.index, n_samples, replace=False)

X_sample = X_test.loc[random_indices]
y_sample = y_test.loc[random_indices]

# Make predictions
predictions = loaded_model.predict(X_sample, verbose=0)
predicted_classes = np.argmax(predictions, axis=1)

# Display results
print("="*80)
print("PREDICTIONS ON RANDOM TEST SAMPLES")
print("="*80)

results = pd.DataFrame({
    'Actual': y_sample.values,
    'Predicted': predicted_classes,
    'Correct': y_sample.values == predicted_classes,
    'Confidence': [f"{max(pred)*100:.1f}%" for pred in predictions]
})

print(results.to_string(index=False))
print("="*80)

accuracy = (results['Correct'].sum() / len(results)) * 100
print(f"\n✅ Accuracy on random samples: {accuracy:.1f}% ({results['Correct'].sum()}/{len(results)} correct)")

### 3.3 - Detailed Prediction Analysis

In [ ]:
# Show detailed predictions for first 5 samples
print("\nDetailed Prediction Breakdown (First 5 samples):")
print("="*80)

for i in range(min(5, len(X_sample))):
    actual = y_sample.iloc[i]
    pred_probs = predictions[i]
    predicted = predicted_classes[i]
    
    print(f"\nSample {i+1}:")
    print(f"  Actual class: {actual} ({['Loss/Draw', 'Win'][actual]})")
    print(f"  Predicted: {predicted} ({['Loss/Draw', 'Win'][predicted]})")
    print(f"  Probabilities: Loss/Draw={pred_probs[0]:.4f}, Win={pred_probs[1]:.4f}")
    print(f"  Result: {'✅ CORRECT' if actual == predicted else '❌ INCORRECT'}")

print("\n" + "="*80)

---
## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

### 4.1 - Experiment with Different Configurations

In [ ]:
# Create an improved model with more layers
improved_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile with custom learning rate
custom_optimizer = Adam(learning_rate=0.001)
improved_model.compile(
    optimizer=custom_optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Improved Model Architecture:")
improved_model.summary()

### 4.2 - Train the Improved Model

In [ ]:
# Train with more epochs
improved_history = improved_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\n✅ Improved model training completed!")

### 4.3 - Evaluate the Improved Model

In [ ]:
# Evaluate improved model
improved_loss, improved_accuracy = improved_model.evaluate(X_test, y_test, verbose=0)

print("="*50)
print("MODEL COMPARISON")
print("="*50)
print(f"Original Model:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"\nImproved Model:")
print(f"  Loss: {improved_loss:.4f}")
print(f"  Accuracy: {improved_accuracy:.4f} ({improved_accuracy*100:.2f}%)")
print("="*50)

if improved_accuracy > test_accuracy:
    print(f"\n✅ Improvement: +{(improved_accuracy - test_accuracy)*100:.2f}% accuracy")
else:
    print(f"\n⚠️  No significant improvement")

### 4.4 - Compare Training Histories

In [ ]:
# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy comparison
ax1.plot(history.history['accuracy'], label='Original - Training', alpha=0.7)
ax1.plot(history.history['val_accuracy'], label='Original - Validation', alpha=0.7)
ax1.plot(improved_history.history['accuracy'], label='Improved - Training', alpha=0.7, linestyle='--')
ax1.plot(improved_history.history['val_accuracy'], label='Improved - Validation', alpha=0.7, linestyle='--')
ax1.set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot loss comparison
ax2.plot(history.history['loss'], label='Original - Training', alpha=0.7)
ax2.plot(history.history['val_loss'], label='Original - Validation', alpha=0.7)
ax2.plot(improved_history.history['loss'], label='Improved - Training', alpha=0.7, linestyle='--')
ax2.plot(improved_history.history['val_loss'], label='Improved - Validation', alpha=0.7, linestyle='--')
ax2.set_title('Loss Comparison', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 📝 Reflection

**Which approach(es) did you find helpful to improve your model performance?**

### Your Answer:

Adding more layers helped the model learn more complex patterns, improving accuracy. Increasing training to 100 epochs made a big difference, allowing the loss to drop significantly. Fine-tuning the learning rate also improved stability. The best configuration was the deeper model with more epochs, which achieved higher accuracy and lower error.